In [ ]:
pip install requests

In [22]:
import requests
import json
import pandas as pd

excel_file = pd.read_excel('withNutrients.xlsx')
column_array = excel_file['Dietary intake'].values

carbs_col = []
calories_col = []
fiber_col = []
fat_col = []
protein_col = []

# Define the range here
# first value is the starting index
# second value is the one after ending index(exclusive)
for i in range(550,600):

    query = column_array[i]
    
    url = 'https://trackapi.nutritionix.com/v2/natural/nutrients'
    headers = {
        #Add the API id and Key here
        'x-app-id': 'e5f0056a',
        'x-app-key': '9043b6a57db369a71107b3671317b765',
        'x-remote-user-id': '0'
    }
    data = {'query': query}

    response = requests.post(url, headers=headers, data=data)
    data = response.json()

    with open('jsonFiles/entry'+str(i)+'.json', 'w') as f:
        json.dump(data, f)
    json_obj = json.loads(json.dumps(data))

    total_carbohydrates = 0
    total_calories = 0
    total_fiber = 0
    total_fat = 0
    total_protein = 0

    if "foods" in json_obj:
        for element in json_obj["foods"]:
            
            total_carbohydrates += (element['nf_total_carbohydrate'] or 0)
            total_calories += (element['nf_calories']or 0)
            total_fiber += (element['nf_dietary_fiber']or 0)
            total_fat += (element['nf_total_fat']or 0)
            total_protein += (element['nf_protein']or 0)


        carbs_col.append(total_carbohydrates)
        calories_col.append(total_calories)
        fiber_col.append(total_fiber)
        fat_col.append(total_fat)
        protein_col.append(total_protein)

        print("----------------Entry Details-  "+str(i)+"  --------------")
        print("")
        print("")
        print(column_array[i])
        print("")
        print("")
        print("carbs : "+str(total_carbohydrates))
        print("calories : "+str(total_calories))
        print("fiber : "+str(total_fiber))
        print("fat : "+str(total_fat))
        print("protein :"+str(total_protein))
        print("")
        print("")


In [25]:
import openpyxl

# Load the existing workbook
workbook = openpyxl.load_workbook("withNutrients.xlsx")

# Get the active worksheet
worksheet = workbook.active

starting_row = 598

for index, value in enumerate(carbs_col, start=starting_row+2):
    worksheet.cell(row=index, column=10, value=value)

for index, value in enumerate(calories_col, start=starting_row+2):
    worksheet.cell(row=index, column=11, value=value)

for index, value in enumerate(fiber_col, start=starting_row+2):
    worksheet.cell(row=index, column=12, value=value)

for index, value in enumerate(fat_col, start=starting_row+2):
    worksheet.cell(row=index, column=13, value=value)

for index, value in enumerate(protein_col, start=starting_row+2):
    worksheet.cell(row=index, column=14, value=value)

workbook.save("withNutrients.xlsx")
